# МОиВС "Генеративные модели", 5-й модуль

# Homework 1

В этой домашней работе вам предстоит добавить к BERT'у декодерную часть и решить задачу генерации суммаризаций для текстов новостей на русском языке.

Дополнительно к этому на отличную оценку потребуется реализовать подсчет метрик качества и менее жадную стратегию выбора следующего токена для генерации.

*Мы сразу вас предостерегаем попасть в петлю бесконечного дообучения модели. Эта домашка не на пробитие скора. Мы будем проверять, что вы, в целом, сделали все верно и смогли получить какую-то более-менее адекватную (такую, которая заметно лучше той, что была до начала обучения) генерацию. Таким образом, если вы видите, что модель учится, не надо дообучать её сутками. Нескольких часов точно должно хватить.*



---


---
По любым вопросам касательно этой домашней работы обращайтесь ко своим ассистентам




In [1]:
%%bash
pip install transformers datasets evaluate bert_score rouge_score

In [2]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, AutoTokenizer

## Подготовка данных (0.5 балла)

Мы воспользуемся датасетом с 🤗 Ильи Гусева "gazeta". Он представляет собой пары (полный текст новости -- его саммари). Пары были взяты с одноименного сайта в домене .ru

Более подробно про датасет можно прочитать [здесь](https://huggingface.co/datasets/IlyaGusev/gazeta)



In [3]:
# Загрузим данные с попощью библиотеки библиотеки datasets

from datasets import load_dataset
dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0", split='train[:5%]')


In [4]:
dataset

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 3048
})

Вы должны помнить, что тексты перед подачей в модель необходимо **токенизировать**.

Добавьте паддинг до `max_length=512` для обучающих данных, а также до `max_length=128` для меток.

Используйте обрезку текстов, длина которых в токенах превышает `max_length`

In [45]:
# Подготовим данные для модели Bert

model_name = 'deepvk/bert-base-uncased' # Указание модели BERT

tokenizer = AutoTokenizer.from_pretrained(model_name)
# special_tokens = {'eos_token': '[EOS]'}
# tokenizer.add_special_tokens(special_tokens)

def preprocess(examples, use_padding=True):
    model_inputs = tokenizer(examples['text'], padding= 'max_length' if use_padding else '', truncation=True, max_length=512)
    summary = tokenizer(examples['summary'], padding= 'max_length' if use_padding else '', truncation=True, max_length=128)
    model_inputs['labels'] = summary['input_ids']
    return model_inputs

/Users/sprilut/miniconda3/envs/ml/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [46]:
tokenized_dataset = dataset.map(preprocess, batched=False)
tokenized_dataset.set_format('torch')

Размер батча советуем подбирать таким образом, чтоб утилизировать максимум доступной VRAM

In [47]:
tokenized_dataset

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3048
})

In [48]:
from torch.utils.data import DataLoader
splitted_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataloader = DataLoader(splitted_dataset['train'], batch_size=8, shuffle=True)
eval_dataloader = DataLoader(splitted_dataset['test'], batch_size=8, shuffle=False)

In [9]:
train_dataloader

In [10]:

# 43, 23, 54 ->
# 1,  0,  0,  0 -> 43, 0, 0
# 1, 43,  0,  0 -> 43, 23, 0
# 1, 43, 23,  0 
# 1, 43, 23, 54

In [11]:
a = torch.tensor([[[1,0,0], [1,1,0]]])
# b = torch.cat([torch.full((a.size()[0], a.size()[1], 1), 100), a[:,:-1]], )

# torch.full((3, 1, 1,), 100), a
# a
# b.T, b.transpose(0,1)
a[:,:,:-1]


tensor([[[1, 0],
         [1, 1]]])

## Реализация Decoder-cети (3 балла)

В данном разделе вам необходимо **реализовать собственный декодер для генерации текста**.

Можете вдохновляться кодом с семинара 1 по GPT. В инициализации весов стоит (но необязательно) проявить смекалку

In [12]:
bert = BertModel.from_pretrained('deepvk/bert-base-uncased')

In [13]:
torch.full([a.size()[0], 1], tokenizer.sep_token_id)

tensor([[2]])

In [52]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import math

# Устанавливаем устройство (GPU, если доступен, иначе CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nn.Transformer
# Класс модели для суммаризации на основе BERT с кастомным декодером
class PositionalEncoding(nn.Module):
    def __init__(self, hidden_size, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, hidden_size, device=device)  # Переносим сразу на устройство
        position = torch.arange(0, max_len, dtype=torch.float, device=device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_size, 2, device=device).float() * (-math.log(10000.0) / hidden_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class BertSummarizer(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', hidden_size=768, num_decoder_layers=3, num_heads=8, dropout=0.1):
        super(BertSummarizer, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name).to(device)  # Переносим модель BERT на устройство
        self.hidden_size = hidden_size
        self.tokenizer = tokenizer
        # Эмбеддинги для токенов на входе в декодер
        self.embedding = nn.Embedding(self.bert.config.vocab_size, hidden_size).to(device)  # Переносим на устройство
        self.positional_encoding = PositionalEncoding(hidden_size)  # Позиционное кодирование также на устройстве
        # Attention головы
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=hidden_size, nhead=num_heads, dropout=dropout, batch_first=True).to(device),
            num_layers=num_decoder_layers,
        ).to(device)  # Переносим декодер на устройство
        self.fc_out = nn.Linear(hidden_size, self.bert.config.vocab_size).to(device)  # Линейный слой на устройство
        self.softmax = nn.Softmax(dim=2).to(device)

    # Функция для создания маски для предотвращения заглядывания вперед в декодере
    def generate_square_subsequent_mask(self, T):
        return torch.triu(
            torch.full((T, T), float('-inf'), device=device, dtype=torch.float64),  # Маска на устройстве
            diagonal=1,
        )

    # def shift_decoder_input(self, input_ids):
    #     pad_column = torch.full([input_ids.size()[0], 1], self.tokenizer.pad_token_id, device=device)  # Перенос на устройство
    #     return torch.cat([input_ids[:, :-1], pad_column,], dim=1)

    def forward(self, input_ids, attention_mask, decoder_input_ids):
        # Переносим данные на устройство
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        decoder_input_ids = decoder_input_ids.to(device)
        encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        memory = encoder_outputs.last_hidden_state  # Выходы BERT для использования в декодере
        
        return self.decoder_forward(decoder_input_ids, memory)

    def decoder_forward(self, input_ids, memory):
        # shifted_ids = self.shift_decoder_input(input_ids)
        embedded = self.embedding(input_ids)
        embedded = self.positional_encoding(embedded)
        decoder_attention_mask = self.generate_square_subsequent_mask(embedded.size(1)).to(device)  # Маска на устройстве
        output = self.decoder(tgt=embedded, memory=memory, tgt_mask=decoder_attention_mask)
        # print('output_size', output.size())
        output = self.fc_out(output)  # Переносим финальный результат на устройство
        # print('output_size', output.size())
        return output

    def generate(self, input_ids, attention_mask, tokenizer, max_len=50):
        # Перенос данных на устройство
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        encoder_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        memory = encoder_outputs.last_hidden_state
        memory.to(device)
        # print('input_ids.size()', input_ids.size())
        # print('encoder_outputs.size()', memory.size())
        batch_size = input_ids.size(0)

        # Начинаем с токена [CLS] или [BOS] (начало последовательности)
        decoder_input_ids = torch.full((batch_size, 1), tokenizer.cls_token_id, dtype=torch.long).to(device)
        
        # memory = memory.transpose(0, 1)

        for _ in range(max_len):
            embedded = self.embedding(decoder_input_ids)
            embedded = self.positional_encoding(embedded)

            decoder_attention_mask = self.generate_square_subsequent_mask(embedded.size(1)).to(device)
            # print('decoder_attention_mask.size()', decoder_attention_mask.size())
            
            decoder_output = self.decoder(tgt=embedded, memory=memory, tgt_mask=decoder_attention_mask)
            # print('decoder_output.size()', decoder_output.size())

            output = self.fc_out(decoder_output)
            # print('output', output.size())

            probs = self.softmax(output)
            # print('probs', probs.size())
            ids = torch.argmax(probs, dim=2)
            # print('ids', ids.size())
            # print('decoder_input_ids', decoder_input_ids.size())
            decoder_input_ids = torch.cat((decoder_input_ids, ids[:, -1:]), dim=1)


            # IndexError: index 0 is out of bounds for dimension 0 with size 0
            if decoder_input_ids[0, -1] == tokenizer.sep_token_id:
                break
        generated_sequence = tokenizer.decode(decoder_input_ids.squeeze().tolist(), skip_special_tokens=True)
        return generated_sequence


In [15]:
eval_data_sample = next(iter(eval_dataloader))
eval_data_sample['labels'][:1]

tensor([[    1,    88, 26278,   520, 31539,  1293, 13095,  2031,   102, 27647,
          4074,    16,  5230, 22881,   524, 10178,  1460, 11041,    18,  4676,
          1389,  5049, 12059,    88,  9686,  4232,  1207,  3280,  7003,    16,
            86,   565,  1088,    16,   617,   848,  9695,  9300,   524,  9535,
            16,   827,  3457, 19814,  1049,   796,  2870,  4225,    18,  1114,
         27647,   298,    16,  3529, 20885,   282,  3045,   532, 23005,  2418,
            16, 14781, 11949,  1013,  8388,   557,    94,  5126,   518,    18,
             2,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,  

In [16]:
# # Инициализируем нашу модель и посморим на ее архитектруру

# model = BertSummarizer(bert_model_name=model_name, tokenizer=tokenizer)
# model = model.to('cuda')
# # model
# eval_data_sample = next(iter(eval_dataloader))

# model.generate(eval_data_sample['input_ids'][:1], eval_data_sample['attention_mask'][:1], tokenizer)
# eval_data_sample['input_ids'].size()

## Обучение модели (1 балл)

<small> 0.25 балла за простейший рабочий цикл; </small>

<small> +0.5 балла за графики для лосса и метрик на трейне и валидации.</small>

В данном разделе вам необходимо **реализовать цикл для обучения модели**


In [17]:
len(train_dataloader), len(eval_dataloader)

(343, 39)

In [64]:
import torch.optim as optim
from tqdm import tqdm  # Для отображения прогресса
import matplotlib.pyplot as plt
from IPython.display import clear_output
import random
# Выбираем устройство: GPU, если доступно, иначе CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)
# Инициализируем модель и переносим её на устройство
model = BertSummarizer(bert_model_name=model_name).to(device)


def shift_decoder_input(input_ids):
    pad_column = torch.full([input_ids.size()[0], 1], tokenizer.pad_token_id, device=device)  # Перенос на устройство
    return torch.cat([input_ids[:, 1:], pad_column], dim=1)

def train_step(model, input_ids, attention_mask, decoder_input_ids, optimizer, criterion, device):
    model.train()
    
    # Перенос данных на устройство
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    # print('decoder_input_ids.size', decoder_input_ids.size())
    decoder_input_ids = decoder_input_ids.to(device)
    labels = decoder_input_ids[:,1:].to(device)
    decoder_input_ids = decoder_input_ids[:,:-1]
    # print('decoder_input_ids.size', decoder_input_ids.size())
    optimizer.zero_grad()  # Обнуляем градиенты
    outputs = model(input_ids, attention_mask, decoder_input_ids)  # Получаем предсказания
    # logits = outputs.reshape(-1, outputs.size(-1)).to(device)
    # labels = labels.reshape(-1).to(device)
    # Вычисляем лосс, учитывая, что output и decoder_input_ids должны быть в одном устройстве
    # print('labels.size', labels.size())

    # outputs = outputs.reshape(-1, outputs.size(-1))
    # labels = labels.reshape(-1)
    logits = outputs.reshape(-1, outputs.size(-1))
    labels = labels.reshape(-1)
    loss = criterion(logits, labels)
    loss.backward()  # Обратное распространение
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()  # Обновление параметров

    return loss.item()

def validate_step(model, input_ids, attention_mask, decoder_input_ids, criterion, device):
    model.eval()
    
    # Перенос данных на устройство
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = decoder_input_ids[:,1:].to(device)
    decoder_input_ids = decoder_input_ids[:,:-1]
    with torch.no_grad():  # Отключаем вычисление градиентов
        outputs = model(input_ids, attention_mask, decoder_input_ids)  # Получаем предсказания
        # outputs = outputs.reshape(-1, outputs.size(-1))
        # Вычисляем лосс для валидации
        logits = outputs.reshape(-1, outputs.size(-1))
        labels = labels.reshape(-1)
        loss = criterion(logits, labels)
        # loss = criterion(outputs.view(-1, outputs.size(-1)), labels.view(-1))

    return loss.item()

# Инициализируем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id).to(device)  # Переносим функцию потерь на устройство
optimizer = optim.Adam(model.parameters())

# Пример данных (используйте DataLoader для реальных данных)
train_data_sample = next(iter(train_dataloader))
val_data_sample = next(iter(eval_dataloader))  # Предполагается, что есть отдельный валидирующий даталоадер

train_losses = []
val_losses = []
num_epochs = 1
plt.ion()  # Включаем интерактивный режим для обновления графика


In [19]:
# model = model.to('cuda')
eval_data_sample = next(iter(eval_dataloader))
model.generate(eval_data_sample['input_ids'][:1], eval_data_sample['attention_mask'][:1], tokenizer)

'##ism 189 ##тере присмот wor закаты кальян инициативу покрас картах отсутствие павлов 1982 коснулся переходов деву фикса целу друж неждан фла окно судя калли ##жик защиту развивается угады ошибаюсь ##гур ##гур ##гур ##гур ##гур ##гур фикса целу ##x6bnm очер http 1940 international ##ple ##ваемых глаз каби фикса целу ##246 917'

## Метрики качества (1 балл)

<small>По 0.33 балла за реализацию каждой из предлагаемых метрик</small>

**Реализуйте функицию для подсчета метрик качества суммаризации.**

Докуметация по некотрым метрикам:
 1. [HuggingFace Rouge](https://huggingface.co/spaces/evaluate-metric/rouge)
 2. [HuggingFace Bleu](https://huggingface.co/spaces/evaluate-metric/bleu)
 3. [HuggingFace BERT Score](https://huggingface.co/spaces/evaluate-metric/bertscore)

In [63]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
bleu = load("bleu")
rouge = load('rouge')

def compute_metrics(predictions, references):
    bleu_score = bleu.compute(predictions=predictions, references=references)
    rouge_score = rouge.compute(predictions=predictions, references=references)
    bertscore_score = bertscore.compute(predictions=predictions, references=references, lang='ru')
    return bleu_score, rouge_score, bertscore_score

def evaluation(model, tokenizer, dataset):
    references = []
    predictions = []
    for i in range(1,len(dataset['input_ids']) + 1):
        predictions.append(model.generate(dataset['input_ids'][i-1:i], dataset['attention_mask'][i-1:i], tokenizer))
        references.append(dataset['summary'][i-1])
    bleu_score, rouge_score, bertscore_score = compute_metrics(predictions, references)
    return bleu_score, rouge_score, bertscore_score


In [21]:
# calculate metrics
bleu_score, rouge_score, bertscore_score = evaluation(model, tokenizer, eval_data_sample)
print(bleu_score, rouge_score, bertscore_score)

/Users/sprilut/miniconda3/envs/ml/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'bleu': 0.0, 'precisions': [0.0016420361247947454, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.75, 'translation_length': 609, 'reference_length': 348} {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0} {'precision': [0.5261016488075256, 0.480069637298584, 0.5033119916915894, 0.49714815616607666, 0.5058144330978394, 0.5059624910354614, 0.4172086715698242, 0.5419792532920837], 'recall': [0.5760972499847412, 0.5489457249641418, 0.5450485944747925, 0.5260737538337708, 0.5561144351959229, 0.5477374792098999, 0.4364463984966278, 0.606736421585083], 'f1': [0.5499655604362488, 0.5122026205062866, 0.5233494639396667, 0.5112020969390869, 0.5297731757164001, 0.5260218977928162, 0.4266107678413391, 0.5725325345993042], 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.44.2)'}


## Обучение модели (0.5 балла)
**Обучите модель, сохраните лучшую версию** (метод `.save_pretrained()` объекта класса AutoModel... или `torch.save()`) **и добавьте пример генерации**. Учтите, что если изменялся токенизатор (а лучше просто по умолчанию), его тоже нужно сохранить. Если планируете продолжить обучение

Для сравнения оценки качества генерации по значениям реализованных метрик можете запустить ruT5-small без дообучения. Мы намеренно даем бейзлайн именно в таком виде.

In [22]:
# Основной цикл обучения
for epoch in tqdm(range(num_epochs), desc="Training Progress"):
    running_train_loss = 0.0
    running_val_loss = 0.0
    
    # Используем tqdm для прогресса по батчам
    batch_iterator = tqdm(train_dataloader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    
    # Тренировка
    for batch_idx, sample in enumerate(batch_iterator):
        # Выполняем один шаг обучения и сохраняем лосс
        loss_item = train_step(
            model,
            sample['input_ids'], 
            sample['attention_mask'], 
            sample['labels'],  # Предполагается, что 'labels' — это целевые токены
            optimizer, 
            criterion,
            device
        )
        running_train_loss += loss_item
        if (batch_idx % 30 == 0):
            train_losses.append(running_train_loss / (batch_idx + 1) )  # Сохраняем текущий лосс

    # Валидация после каждой эпохи
    model.eval()  # Переводим модель в режим валидации
    total_val_loss = 0.0
    for val_batch_idx, val_sample in enumerate(eval_dataloader):
        val_loss_item = validate_step(
            model,
            val_sample['input_ids'],
            val_sample['attention_mask'],
            val_sample['labels'],  # Предполагается, что 'labels' — это целевые токены для валидации
            criterion,
            device
        )
        total_val_loss += val_loss_item
    val_loss = total_val_loss / len(eval_dataloader)
    val_losses.append(val_loss)
    bleu_score, rouge_score, bertscore_score = evaluation(model, tokenizer, eval_data_sample)

    clear_output(wait=True)  # Очищаем старый график
    plt.figure(figsize=(12, 8))
    
    # График для лоссов
    plt.subplot(2, 1, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss', linestyle='--')
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.title(f'Training and Validation Loss (Epoch {epoch+1})')
    plt.grid(True)
    plt.legend()

    # График для метрик
    plt.subplot(2, 1, 2)
    plt.bar(['BLEU', 'ROUGE-L', 'BERTScore'], [bleu_score['bleu'], rouge_score['rougeL'], np.mean(bertscore_score['f1'])])
    plt.title('Evaluation Metrics')
    plt.ylabel('Score')

    plt.tight_layout()
    plt.show()

    # Средний лосс за эпоху
    epoch_train_loss = running_train_loss / len(train_dataloader)
    epoch_val_loss = val_loss

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}")
    print(f"BLEU: {bleu_score['bleu']:.4f}, ROUGE-L: {rouge_score['rougeL']:.4f}, BERTScore: {np.mean(bertscore_score['f1']):.4f}")


plt.ioff()  # Отключаем


In [24]:
# Сохранение модели и токенизатора
model_save_path = 'my_summarizer_model.pt'
torch.save(model.state_dict(), model_save_path)

loaded_model = BertSummarizer(bert_model_name=model_name)
loaded_model.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

## Реализация менее жадных стратегий выбора следующего токена (4 балла)
Всегда ли выбор наиболее вероятного токена на каждом шаге – это лучшая стратегия для генерации текста?

<details>
    <summary>Спойлер</summary>
    <p>Нет</p>
</details>

**Сравнение стратегий для генерации текста:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br><br/> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br><br/> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words <br/> that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br><br/> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps <br/> the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br><br/> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br><br/> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities <br/> by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br><br/> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br><br/> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br><br/> **Cons:** Computationally more complex and requires a good loss function. |

Ссылки на докуметацию:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

**1. Дополните метод `generate` в модели, чтобы получать топ-k самых вероятных токена и их "вероятности"** (1 балл).   

**2. Реализуйте стратегию Nucleus Sampling в методе `generate`** (1 балл)

**3. Реализуйте стратегию Beam Search** (2 балла)

Получилось ли улучшить генерацию?

In [60]:
import torch
import torch.nn.functional as F

class BertSummarizerBase(BertSummarizer):
    def __init__(self, *args, **kwargs):
        super(BertSummarizerBase, self).__init__(*args, **kwargs)
    def generate(self, input_ids, attention_mask, tokenizer, max_length=100):
        batch_size = input_ids.shape[0]
        generated = [[] for _ in range(batch_size)]
        
        decoder_input_ids = torch.full((batch_size, 1), tokenizer.cls_token_id, dtype=torch.long, device=input_ids.device)
        for _ in range(max_length):
            with torch.no_grad():
                outputs = self.forward(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)
            next_token_logits = outputs[:, -1, :]
            
            next_tokens = self.get_next_token(next_token_logits)
            
            for i, token in enumerate(next_tokens.squeeze(1)):
                generated[i].append(token.item())
            
            decoder_input_ids = torch.cat([decoder_input_ids, next_tokens], dim=-1)
            
            if all(tokenizer.eos_token_id in seq for seq in generated):
                break
        
        return [tokenizer.decode(seq, skip_special_tokens=True) for seq in generated]

    def get_next_token(self, logits):
        raise NotImplementedError("This method should be implemented in derived classes.")

class BertSummarizerGreedy(BertSummarizerBase):
    def get_next_token(self, logits):
        return torch.argmax(logits, dim=-1).unsqueeze(1)

class BertSummarizerTopK(BertSummarizerBase):
    def __init__(self, *args, k=5, **kwargs):
        super(BertSummarizerTopK, self).__init__(*args, **kwargs)
        self.k = k

    def get_next_token(self, logits):
        top_k_logits, top_k_indices = torch.topk(logits, self.k, dim=-1)
        top_k_probs = F.softmax(top_k_logits, dim=-1)
        return top_k_indices[:, 0].unsqueeze(1)

class BertSummarizerNucleusSampling(BertSummarizerBase):
    def __init__(self, *args, p=0.9, **kwargs):
        super(BertSummarizerNucleusSampling, self).__init__(*args, **kwargs)
        self.p = p

    def get_next_token(self, logits):
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
        sorted_indices_to_remove = cumulative_probs > self.p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        filtered_logits = torch.where(indices_to_remove, torch.ones_like(logits) * float('-inf'), logits)
        probabilities = F.softmax(filtered_logits, dim=-1)
        return torch.multinomial(probabilities, 1)


In [62]:
len(eval_dataloader)

39

In [61]:

from evaluate import load
from tqdm import tqdm
import matplotlib.pyplot as plt

# Функция для генерации саммари
def generate_summaries(model, dataloader):
    summaries = []
    for batch in tqdm(dataloader, desc="Generating summaries"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        generated = model.generate(input_ids, attention_mask, tokenizer, max_length=50)
        summaries.extend(generated)
    return summaries

# Загрузка и оценка базовой модели
model_base = BertSummarizerGreedy(bert_model_name=model_name)
model_base.load_state_dict(torch.load(model_save_path))
summaries_base = generate_summaries(model_base, eval_dataloader)
# references = [example['summary'] for example in eval_dataloader[:len(summaries_base)]]
scores_base = compute_metrics(summaries_base, references)

# print("Базовая модель:")
# print(scores_base)

# # Загрузка и оценка модели с Top-K сэмплированием
# model_top_k = BertSummarizerTopK(bert_model_name=model_name)
# model_top_k.load_state_dict(torch.load(model_save_path))
# summaries_top_k = generate_summaries(model_top_k, eval_dataloader)
# scores_top_k = compute_metrics(summaries_top_k, references)

# print("\nМодель с Top-K сэмплированием:")
# print(scores_top_k)

# # Загрузка и оценка модели с Nucleus сэмплированием
# model_nucleus = BertSummarizerNucleusSampling(bert_model_name=model_name)
# model_nucleus.load_state_dict(torch.load(model_save_path))
# summaries_nucleus = generate_summaries(model_nucleus, eval_dataloader)
# scores_nucleus = compute_metrics(summaries_nucleus, references)

# print("\nМодель с Nucleus сэмплированием:")
# print(scores_nucleus)

# # Сравнение результатов
# print("\nСравнение результатов:")
# for metric in ['rouge1', 'rouge2', 'rougeL']:
#     print(f"{metric}:")
#     print(f"  Базовая модель: {scores_base[metric]:.4f}")
#     print(f"  Top-K: {scores_top_k[metric]:.4f}")
#     print(f"  Nucleus: {scores_nucleus[metric]:.4f}")

# # Создание столбчатой диаграммы
# metrics = ['rouge1', 'rouge2', 'rougeL']
# models = ['Базовая модель', 'Top-K', 'Nucleus']

# fig, ax = plt.subplots(figsize=(12, 6))

# x = np.arange(len(metrics))
# width = 0.25

# ax.bar(x - width, [scores_base[m] for m in metrics], width, label='Базовая модель')
# ax.bar(x, [scores_top_k[m] for m in metrics], width, label='Top-K')
# ax.bar(x + width, [scores_nucleus[m] for m in metrics], width, label='Nucleus')

# ax.set_ylabel('Значение метрики')
# ax.set_title('Сравнение метрик ROUGE для разных моделей')
# ax.set_xticks(x)
# ax.set_xticklabels(metrics)
# ax.legend()

# plt.tight_layout()
# plt.show()

Generating summaries:   0%|          | 0/39 [00:00<?, ?it/s]

input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8

Generating summaries:   3%|▎         | 1/39 [01:14<47:29, 74.99s/it]

input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8, 512])
input_ids.size torch.Size([8

Generating summaries:   3%|▎         | 1/39 [01:57<1:14:12, 117.16s/it]


KeyboardInterrupt: 

## Послевкусие (0 баллов)

Если эта домашняя работа показалась вам недостаточно большой, предлагаем провести следующий эксперимент:

- от имеющейся модели "откусить" только декодерную часть (откусить также можно от ruT5-small);
- немного дообучить (что называется, по вкусу);
- посмотреть качество генерации по метрикам и "глазами";
- сравнить полученное с Encoder-Decoder архитектурой;
- ответить на вопрос "Дает ли применение Encoder-Decoder архитектуры значительный буст в качестве генерации, или это некоторый overkill?" (базово, ответ лежит на поверхности 😸)

Ещё более опционально можно:
- почитать про возможности генерации Encoder-only архитектурными решениями (BERT, e.g.)
- сравнить с генерацией только Decoder'ом и both Encoder-Decoder'ом;
- в т.ч. подобрать число обучаемых параметров таким образом, чтоб оно было примерно одинаковым для каждого инстанса моделей (их, инстансов, будет 3 -- только энкодер, только декодер и энкодер-декодер).

*Вообще ориентироваться следует на следующее утверждение: "Только энкодерные архитектуры (BERT, e.g.) хороши для понимания текста (получения эмеддингов), лишь декодерные (GPT, например) -- для генерации, энкодер-декодерные (скажем, T5) -- для обеих задач"*